# Hunter Participation by State (WIP)

A look at trends of hunter participation by state and region in the United States based on hunting license information from 2004 to 2018.

## Data Source

Hunting license data are from U.S. Fish and Wildlife Service's [Wildlife and Sport Fish Restoration Program](https://wsfrprograms.fws.gov/Subpages/LicenseInfo/Hunting.htm) page.

In [15]:
import os
import json

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

In [16]:
# import sys
# !{sys.executable} -m pip install folium

In [17]:
import folium

## Import the Data from the Summary Excel Workbook

The data for each year (FY2004-FY2018) include the following columns:

- Paid Hunting License Holders: this is one individual regardless of the number of licenses purchased
- The number of Resident Hunting Licenses, Tags, Permits, and Stamps
- The number of Non-Resident Hunting Licenses, Tags, Permits, and Stamps
- Total Hunting Licenses, Tags, Permits, and Stamps (sum of the previous two items). Persons who hunted in more than one state are counted in each state where they hunted
- Gross Cost - Hunting Licenses (in US$)

Years FY2016-FY2018 also include the following columns to break out license costs by resident or non-resident:

- Cost - Resident Hunting Licenses, Tags, Permits, and Stamps
- Cost - Non-Resident Hunting Licenses, Tags, Permits, and Stamps

One item to note: the total license data for Rhode Island has a missing value in 2011 and some odd fluctuations in the following years (2011-2016), alternating between numbers in the eight thousands to the thirty thousands. For this reason, the choropleth maps are based on the growth rates for paid hunting license holders.

In [18]:
# Excel spreadsheet path
excel_path = './data/NatlHuntingLicenseReportAllYearsClean.xlsx'
index_col = 'State'

# Import the number of paid license holders by state (one individual regardless of licenses purchased)
license_holders = pd.read_excel(excel_path,
                                sheet_name='LicenseHolders',
                                index_col=index_col)
license_holders['2003-18 Change (%)'] = (license_holders[2018] - license_holders[2003]) / license_holders[2003] *100
license_holders['2008-18 Change (%)'] = (license_holders[2018] - license_holders[2008]) / license_holders[2008] *100
license_holders['2013-18 Change (%)'] = (license_holders[2018] - license_holders[2013]) / license_holders[2013] *100

In [19]:
license_holders.head()

,Region,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2003-18 Change (%),2008-18 Change (%),2013-18 Change (%)
State,,,,,,,,,,,,,,,,,,,,
AK,Pacific,108921,108487,106916,107131,107260,101547,106653,101750,97858,96979,96979,99954,98084,99121,97537,99121,9.886906,12.314006,7.261662
AL,East South Central,547905,548829,565139,507926,507403,530127,527713,540098,264710,264640,255923,259241,267354,270229,273638,270229,102.755811,114.089785,3.353536
AR,West South Central,326559,340200,328542,326779,382212,390554,385275,390932,381521,380931,354042,375737,378162,386559,402493,386559,-15.521563,-7.762638,-16.385698
AZ,Mountain,305214,324553,215444,200092,195213,195664,194727,191834,201627,202976,202976,196706,182044,181467,183478,181467,68.192564,50.369502,55.988838
CA,Pacific,280967,284069,287147,283539,283539,281472,289609,293263,296623,300615,295163,299303,305962,315588,320092,315588,-10.970316,-4.809546,-0.179414


In [21]:
# Import the number of total licenses by state
# Persons who hunted in more than one state are counted in each state where they hunted
total_licenses = pd.read_excel(excel_path,
                               sheet_name='TotalLicenses',
                               index_col=index_col)
total_licenses['2003-18 Year Change (%)'] = (total_licenses[2018] - total_licenses[2003]) / total_licenses[2003] *100
total_licenses['2008-18 Year Change (%)'] = (total_licenses[2018] - total_licenses[2008]) / total_licenses[2008] *100
total_licenses['2013-18 Year Change (%)'] = (total_licenses[2018] - total_licenses[2013]) / total_licenses[2013] *100

In [22]:
total_licenses.tail()

,Region,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2003-18 Year Change (%),2008-18 Year Change (%),2013-18 Year Change (%)
State,,,,,,,,,,,,,,,,,,,,
VT,New England,169328,177428,178743,183780,172351,189241,191735,188068,172964,172964,165583,154596,170680,186789,192308,186789,-9.347981,2.261706,-10.522561
WA,Pacific,691732,685166,674196,676025,666678,738385,770206,775028,763171,782009,837738,929616,902485,883616,791512,883616,-21.715768,-17.428599,-6.318249
WI,East North Central,2863676,2836703,2941239,2968341,2880270,2941549,3041084,2972072,2887881,3072436,3072436,3143230,3030609,2717877,2599506,2717877,5.364444,-6.794609,-2.647347
WV,South Atlantic,677710,705072,719105,704055,684738,710174,778357,767999,736383,753449,753449,738049,949284,926177,830748,926177,-26.827162,-10.052306,-4.571274
WY,Mountain,247914,240862,249478,261991,264957,269424,265887,260973,278632,266483,266483,249489,242143,228394,240898,228394,8.546634,-6.968174,-7.983699


## 15-Year Growth or Decline of License Holders by State

In [28]:
fifteen_yr_holders = license_holders['2003-18 Change (%)']  #.sort_values(by='2003-18 Change (%)')
print('Minimum growth: {}'.format(fifteen_yr_holders.min()))
print('Maximum growth: {}'.format(fifteen_yr_holders.max()))
print(fifteen_yr_holders.sort_values())

Minimum growth: -31.74634963405309
Maximum growth: 102.75581081231104
State
CT    -31.746350
RI    -23.215789
SC    -22.871693
VT    -22.380424
ME    -18.865345
MI    -18.270239
WV    -17.391900
IA    -17.018445
AR    -15.521563
DE    -11.058507
CA    -10.970316
OR    -10.404777
NY     -9.746217
VA     -9.384613
NH     -9.184838
OH     -8.386073
NJ     -8.229347
MO     -8.140069
WA     -5.787204
WY     -5.355288
CO     -4.997402
PA     -4.141473
TN     -3.700904
ND     -3.052531
MD     -2.720313
NM     -2.380216
WI     -1.368333
MN     -0.935957
KY      1.447698
MA      2.221948
NE      3.713272
IL      4.091920
FL      7.890200
MT      9.099520
AK      9.886906
TX     11.396991
SD     12.137152
NV     13.056492
IN     14.873357
ID     16.950334
HI     26.573677
MS     27.479220
KS     28.342710
NC     35.111708
LA     48.369935
UT     52.795641
OK     59.691201
AZ     68.192564
GA     96.479754
AL    102.755811
Name: 2003-18 Change (%), dtype: float64


In [38]:
# Create choropleth map

us_states = os.path.join('Data', 'us-states.json')
geo_json_data = json.load(open(us_states))

m = folium.Map(location=[43, -102], zoom_start=4)

threshold_scale = np.quantile(fifteen_yr_holders, [0, 0.2, 0.4, 0.6, 0.8, 1])

folium.Choropleth(
                  geo_data=geo_json_data,
                  data=ten_yr_holders,
                  name='Choropleth',
                  key_on='feature.id',
                  fill_color='RdYlGn',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  line_weight=1,
                  legend_name='15 Year Change in Hunting License Holders (States divided 10 per color)',
                  threshold_scale=threshold_scale,
                  highlight=True,
                 ).add_to(m)

folium.LayerControl().add_to(m)
m

## Paid Hunting License Holder Declines in New England

To come.

In [23]:
# New England states subset
# This uses paid license holders (individual hunters) vs total licenses
#   because the latter set had missing data in RI for FY2013
ne_states = ['CT', 'ME', 'MA', 'NH', 'RI', 'VT']
ne_hunters = license_holders.loc[ne_states]
ne_hunters

,Region,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2003-18 Change (%),2008-18 Change (%),2013-18 Change (%)
State,,,,,,,,,,,,,,,,,,,,
CT,New England,37489,39488,42924,42535,41294,44178,46582,50066,49493,48857,48857,52207,54130,54926,57720,54926,-31.746350,-23.267904,-15.141020
ME,New England,163191,166051,168890,165781,175196,189120,191280,193436,196160,195568,195568,199102,205600,201136,197908,201136,-18.865345,-16.555367,-13.710343
MA,New England,57921,57973,59669,56797,61204,57641,57346,59470,57153,59158,57193,59016,69500,56662,68488,56662,2.221948,1.272883,0.485765
NH,New England,58099,59318,61556,59068,56411,59301,59420,59154,59768,61076,61076,60737,62587,63975,67586,63975,-9.184838,-4.874255,-2.026947
RI,New England,8209,8797,8978,8624,8551,8605,8798,8858,9075,9075,9075,8940,9302,10691,10750,10691,-23.215789,-9.542700,-4.601976
VT,New England,69943,71807,74219,72930,74966,80650,82307,83681,84044,84044,83708,81265,86512,90110,92138,90110,-22.380424,-16.444067,-13.275883


In [75]:
# Create New England choropleth map

# us_states = os.path.join('Data', 'us-states.json')
# geo_json_data = json.load(open(us_states))

m_ne = folium.Map(location=[44, -73], zoom_start=6)

threshold_scale = np.quantile(fifteen_yr_holders, [0, 0.2, 0.4, 0.6, 0.8, 1])

folium.Choropleth(
                  geo_data=geo_json_data,
                  data=ten_yr_holders,
                  name='15 Year Decline (%)',
                  key_on='feature.id',
                  fill_color='RdYlGn',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  line_weight=1,
                  legend_name='15 Year Change in Hunting License Holders',
                  threshold_scale=threshold_scale,
                  highlight=True,
                 ).add_to(m_ne)


folium.LayerControl(collapsed=False).add_to(m_ne)
m_ne

## Hunting License Cost Differential between Residents and Non-Residents

Typically, residents enjoy better hunting access in their respective states compared non-residents in two ways. Resident pay less for hunting licenses, tags, permits, and stamps versus what non-residents pay, and they tend to claim larger percentages for any tags that aren't available over-the-counter (one that must be drawn).

The general argument for why residents deserve more and cheaper access is that wildlife management is the state's responsibility - both to execute and fund at the state level. There are several major sources of cash that fund these departments (see the [Conservation Funding project](../ConservationFunding/ConservationFundingSources.ipynb) for a general overview), one of which is the money collected from hunting licenses, tags, permits, and stamps.

However, it appears that for FY2018, 14 states actually collected a total dollar amount more from non-residents than they did from residents. These states are:

- Alaska
- Alabama
- Arkansas
- Colorado
- Georgia
- Idaho
- Kansas
- Mississippi
- Montana
- North Dakota
- New Mexico
- South Dakota
- West Virginia
- Wyoming

In the table below, the "Non-resident Cost Factor" column shows the multiple of how many times over residents paid vs. non-residents. Negative numbers indicate where non-residents paid a factor higher than residents. For example, Alaska shows a cost factor of -1.9x, which means non-residents paid 1.9 times more into the system for hunting licenses, tags, permits, and stamps than residents did. For Arizona, it's the opposite - the cost factor is almost 3.0x, meaning residents paid into the system three times more than non-residents.

Looking at the total hunting dollars collected from residents and non-residents, California had the highest ratio from residents, at 20.5x. So for every \$1.00 a non-resident paid for hunting licenses, etc., a resident contributed \$20.50. On the other end of the spectrum, Colorado collected the highest ratio from non-residents, at -3.5x. This means that for every \$1.00 a resident put into their system, a non-resident contributed \$3.50.

In [39]:
# Get 2018 Data
data_2018 = pd.read_excel(excel_path,
                          sheet_name='2018-Full',
                          index_col=index_col)

In [40]:
# Calculate the resident vs non-resident cost factor differential
res_col = '2018 Cost - Resident Hunting Licenses, Tags, Permits, and Stamps'
non_res_col = '2018 Cost - Non-Resident Hunting Licenses, Tags, Permits, and Stamps'
new_col = 'Non-Resident Cost Factor'

factor_mask = data_2018[res_col] >= data_2018[non_res_col]
data_2018_valid = data_2018[factor_mask]
data_2018_notvalid = data_2018[~factor_mask]

data_2018[new_col] = 0

data_2018.loc[factor_mask, new_col] = data_2018_valid[res_col] / data_2018_valid[non_res_col]
data_2018.loc[~factor_mask, new_col] = data_2018_notvalid[non_res_col] / data_2018_notvalid[res_col] * -1

data_2018.head()

,2018 Paid Hunting License Holders,"2018 Resident Hunting Licenses, Tags, Permits, and Stamps","2018 Non-Resident Hunting Licenses, Tags, Permits, and Stamps","2018 Total Hunting Licenses, Tags, Permits, and Stamps","2018 Cost - Resident Hunting Licenses, Tags, Permits, and Stamps","2018 Cost - Non-Resident Hunting Licenses, Tags, Permits, and Stamps",2018 Gross Cost - Hunting Licenses,Non-Resident Cost Factor
State,,,,,,,,
AK,108921,278248,74532,352780,2922692,5568901,8491593,-1.905401
AL,547905,542838,37106,579944,5693734,6107717,11801451,-1.072709
AR,326559,380102,136944,517046,8186097,10579420,18765517,-1.292364
AZ,305214,456140,61607,517747,13358303,4515780,17874083,2.958139
CA,280967,1007422,22738,1030160,20609712,1006282,21615994,20.481050


In [41]:
# Calculate the stats for Resident vs. Non-Resident Costs
neg_states = data_2018[data_2018['Non-Resident Cost Factor'] < 0]
mega_res = max(data_2018['Non-Resident Cost Factor'])
mega_res_state = data_2018['Non-Resident Cost Factor'].idxmax(mega_res)
mega_nonres = min(data_2018['Non-Resident Cost Factor'])
mega_nonres_state = data_2018['Non-Resident Cost Factor'].idxmin(mega_nonres)

print('Number of states collecting more license fees from non-residents: {}'.format(len(neg_states)))
print('State with the highest resident contribution factor: {0} with {1:0.1f}x'.format(mega_res_state, mega_res))
print('State with the highest non-resident contribution factor: {0} with {1:0.1f}x'.format(mega_nonres_state, mega_nonres))

Number of states collecting more license fees from non-residents: 14
State with the highest resident contribution factor: CA with 20.5x
State with the highest non-resident contribution factor: CO with -3.5x


In [14]:
data_2018

,2018 Paid Hunting License Holders,"2018 Resident Hunting Licenses, Tags, Permits, and Stamps","2018 Non-Resident Hunting Licenses, Tags, Permits, and Stamps","2018 Total Hunting Licenses, Tags, Permits, and Stamps","2018 Cost - Resident Hunting Licenses, Tags, Permits, and Stamps","2018 Cost - Non-Resident Hunting Licenses, Tags, Permits, and Stamps",2018 Gross Cost - Hunting Licenses,Non-Resident Cost Factor
State,,,,,,,,
AK,108921,278248,74532,352780,2922692,5568901,8491593,-1.905401
AL,547905,542838,37106,579944,5693734,6107717,11801451,-1.072709
AR,326559,380102,136944,517046,8186097,10579420,18765517,-1.292364
AZ,305214,456140,61607,517747,13358303,4515780,17874083,2.958139
CA,280967,1007422,22738,1030160,20609712,1006282,21615994,20.481050
CO,294319,459518,109246,568764,12751122,44305473,57056595,-3.474633
CT,37489,124169,4063,128232,2085367,394950,2480317,5.280078
DE,17847,52315,7125,59440,632742,372462,1005204,1.698810
FL,190232,313554,18155,331709,7890888,984031,8874919,8.018942


In [60]:
# Separate factor data to create the choropleth map
res_data_2018 = data_2018['Non-Resident Cost Factor']

# Export data as JSON for potential use in other mapping programs
with open('./Data/StateResVNonResCostFactor.json', 'w') as json_file:
    json_file.write(json.dumps(dict(res_data_2018)))

res_data_2018.head()

State
AK    -1.905401
AL    -1.072709
AR    -1.292364
AZ     2.958139
CA    20.481050
Name: Non-Resident Cost Factor, dtype: float64

In [61]:
# Create choropleth map

m_res = folium.Map(location=[43, -102], zoom_start=4)

threshold_scale = [-3.5, -1.5, 0, 2, 5, 20.5]
# threshold_scale = np.quantile(res_data_2018, [0, 0.2, 0.4, 0.6, 0.8, 1])

folium.Choropleth(
                  geo_data=geo_json_data,
                  data=res_data_2018,
                  name='Choropleth',
                  key_on='feature.id',
                  fill_color='RdYlGn',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  line_weight=1,
                  legend_name='Cost Differential bn Residents and Non-Residents (States divided 10 per color)',
                  threshold_scale=threshold_scale,
                  highlight=True,
                 ).add_to(m_res)

folium.LayerControl().add_to(m_res)
m_res